# Basic LLM Usage

This notebook demonstrates basic usage of Hugging Face Transformers for generation, classification, and translation. Designed to run on local machine or Colab.

In [ ]:
# Install required packages (uncomment when running in a fresh environment)
# !pip install transformers accelerate sentencepiece

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

print('transformers version loaded')

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# 1. Generate some sample data
X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(-1, 1) # Features
y = np.array([2, 4, 5, 4, 5, 7, 8, 9, 10, 12]) # Target variable

# 2. Define the model
model = LinearRegression()

# 3. Train the model
model.fit(X, y)

# 4. Make a prediction
new_X = np.array([11]).reshape(-1, 1)
prediction = model.predict(new_X)

print(f"Trained model coefficient: {model.coef_[0]:.2f}")
print(f"Trained model intercept: {model.intercept_:.2f}")
print(f"Prediction for new data {new_X[0][0]}: {prediction[0]:.2f}")

In [ ]:
# pip install scikit-learn

## 1.text-classification

In [ ]:
from transformers import pipeline

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification")

In [ ]:
data = pipe("This restaurant is not awesome")
data[0]['label']


⭐ 3.6 Zero-Shot Classification

In [ ]:
pipe = pipeline("zero-shot-classification")

result = pipe(
    "This is a course about AI and machine learning.",
    candidate_labels=["education", "politics", "sports"]
)

print(result)


https://huggingface.co/docs/transformers/v4.57.0/en/main_classes/pipelines#transformers.Pipeline.model



## 2.Text Generations

In [ ]:
pipe = pipeline("text-generation", model="gpt2")

result = pipe(
    "Once upon a time in India, there was a data scientist",
    max_length=50,
    num_return_sequences=1
)

print(result)


In [ ]:
from transformers import pipeline

pipeline = pipeline(task="text-generation", model="google/gemma-2-2b")
pipeline("the secret to baking a really good cake is ")
[{'generated_text': 'the secret to baking a really good cake is 1. the right ingredients 2. the'}]

## 3. text summarizations

In [ ]:
from transformers import pipeline

pipeline = pipeline(task="summarization", model="google/pegasus-billsum")

In [ ]:
summarize_text = pipeline("Section was formerly set out as section 44 of this title. As originally enacted, this section contained two further provisions that 'nothing in this act shall be construed as in any wise affecting the grant of lands made to the State of California by virtue of the act entitled 'An act authorizing a grant to the State of California of the Yosemite Valley, and of the land' embracing the Mariposa Big-Tree Grove, approved June thirtieth, eighteen hundred and sixty-four; or as affecting any bona-fide entry of land made within the limits above described under any law of the United States prior to the approval of this act.' The first quoted provision was omitted from the Code because the land, granted to the state of California pursuant to the Act cite, was receded to the United States. Resolution June 11, 1906, No. 27, accepted the recession.")
[{'summary_text': 'Instructs the Secretary of the Interior to convey to the State of California all right, title, and interest of the United States in and to specified lands which are located within the Yosemite and Mariposa National Forests, California.'}]


summarize_text

In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis")

result = pipe("I love using Hugging Face pipelines!")
print(result)

⭐ 3.5 Translation (English → Hindi)

In [ ]:

pipe = pipeline("translation_en_to_hi", model="Helsinki-NLP/opus-mt-en-hi")
print(pipe("I love learning transformers!"))

⭐ 3.7 Image Classification

In [ ]:
pipe = pipeline("image-classification", model="google/vit-base-patch16-224")

from PIL import Image
img = Image.open("/content/new_pic.jpg")

print(pipe(img))


In [ ]:
# ⭐ 3.8 Speech-to-Text (ASR)
pipe = pipeline("automatic-speech-recognition",
                model="openai/whisper-small")


# print(pipe("audio.wav"))
# https://huggingface.co/docs/transformers/en/model_doc/whisper

In [ ]:
pipe("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

In [ ]:
# ⭐ 3.9 Embeddings / Feature Extraction
pipe = pipeline("feature-extraction",
                model="sentence-transformers/all-MiniLM-L6-v2")

emb = pipe("This is a feature extraction example.")
print(len(emb[0]))   # 384 dims


In [ ]:
 # hf_ CXeDvBPImWhKBaiJ
    # BFfuuKpqlQKxjIgIIA


## ⭐ 3.10 Text-to-Image

In [ ]:
# ⭐ 3.10 Text-to-Image

# (Pipeline does NOT support; must use Diffusers)

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

In [ ]:
image = pipe("a futuristic city in neon lights").images[0]
image.save("city.png")

In [ ]:
3

Nice, let’s turn all those Hugging Face models into **one Streamlit app with a dropdown menu** 😄

Below is a **single `app.py`** file where you can:

* Pick a **task** from a dropdown.
* See task-specific inputs (text, labels, image upload, audio upload).
* Run the corresponding **Hugging Face pipeline / diffusers model**.
* View the result directly in the browser.

---

## 🔧 1. `requirements.txt`

Create a `requirements.txt`:

```txt
streamlit
transformers
torch
diffusers
sentencepiece
accelerate
safetensors
Pillow
```

Install:

```bash
pip install -r requirements.txt
```

---

## 🧠 2. `app.py` – Streamlit Multi-Model Demo

```python
import streamlit as st
from transformers import pipeline
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import tempfile
import os

st.set_page_config(page_title="Hugging Face Multi-Model Demo", layout="wide")
st.title("🤗 Hugging Face Multi-Model Playground (Streamlit)")

st.sidebar.header("⚙️ Settings")
task = st.sidebar.selectbox(
    "Choose a task",
    [
        "Sentiment Analysis",
        "Text Classification",
        "Text Generation",
        "Summarization",
        "Translation EN→HI",
        "Zero-shot Classification",
        "Image Classification",
        "Speech to Text",
        "Feature Extraction (Embeddings)",
        "Text to Image (Stable Diffusion)",
    ],
)

st.sidebar.markdown("---")
st.sidebar.write("🧠 Models are loaded lazily and cached.")
st.sidebar.write("💡 First run of a task may take time (downloads model).")


# -------------------- CACHED PIPELINES -------------------- #

@st.cache_resource
def get_sentiment_pipe():
    return pipeline("sentiment-analysis")

@st.cache_resource
def get_text_cls_pipe():
    return pipeline("text-classification", model="distilbert-base-uncased")

@st.cache_resource
def get_text_gen_pipe():
    return pipeline("text-generation", model="gpt2")

@st.cache_resource
def get_summarization_pipe():
    return pipeline("summarization", model="facebook/bart-large-cnn")

@st.cache_resource
def get_translation_pipe():
    return pipeline("translation_en_to_hi", model="Helsinki-NLP/opus-mt-en-hi")

@st.cache_resource
def get_zero_shot_pipe():
    return pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

@st.cache_resource
def get_image_cls_pipe():
    return pipeline("image-classification", model="google/vit-base-patch16-224")

@st.cache_resource
def get_asr_pipe():
    # Small Whisper model for demo
    return pipeline("automatic-speech-recognition", model="openai/whisper-small")

@st.cache_resource
def get_embedding_pipe():
    return pipeline(
        "feature-extraction",
        model="sentence-transformers/all-MiniLM-L6-v2"
    )

@st.cache_resource
def get_sd_pipe():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16 if device == "cuda" else torch.float32
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=dtype,
    ).to(device)
    return pipe, device


# -------------------- TASK UIs -------------------- #

if task == "Sentiment Analysis":
    st.subheader("🧪 Sentiment Analysis")
    text = st.text_area("Enter text", "I love using Hugging Face with Streamlit!")

    if st.button("Analyze Sentiment"):
        if text.strip():
            pipe = get_sentiment_pipe()
            result = pipe(text)
            st.json(result)
        else:
            st.warning("Please enter some text.")

elif task == "Text Classification":
    st.subheader("🏷️ Text Classification")
    text = st.text_area("Enter text", "This course is amazing and very helpful.")

    if st.button("Classify"):
        if text.strip():
            pipe = get_text_cls_pipe()
            result = pipe(text)
            st.json(result)
        else:
            st.warning("Please enter some text.")

elif task == "Text Generation":
    st.subheader("✍️ Text Generation")
    prompt = st.text_area("Enter prompt", "Once upon a time in India, there was a data scientist")
    max_len = st.slider("Max length", 20, 200, 60)
    num_return = st.slider("Number of sequences", 1, 5, 1)

    if st.button("Generate Text"):
        if prompt.strip():
            pipe = get_text_gen_pipe()
            outputs = pipe(
                prompt,
                max_length=max_len,
                num_return_sequences=num_return,
                do_sample=True,
                top_k=50,
                top_p=0.95,
            )
            for i, out in enumerate(outputs):
                st.markdown(f"**Output {i+1}:**")
                st.write(out["generated_text"])
        else:
            st.warning("Please enter a prompt.")

elif task == "Summarization":
    st.subheader("📝 Summarization")
    text = st.text_area("Enter long text to summarize", height=200)

    max_len = st.slider("Max summary length", 20, 200, 100)
    min_len = st.slider("Min summary length", 5, 100, 30)

    if st.button("Summarize"):
        if text.strip():
            pipe = get_summarization_pipe()
            result = pipe(
                text,
                max_length=max_len,
                min_length=min_len,
                do_sample=False
            )
            st.subheader("Summary")
            st.write(result[0]["summary_text"])
        else:
            st.warning("Please enter some text.")

elif task == "Translation EN→HI":
    st.subheader("🌐 Translation (English → Hindi)")
    text = st.text_area("Enter English text", "I love learning Transformers and Generative AI.")

    if st.button("Translate"):
        if text.strip():
            pipe = get_translation_pipe()
            result = pipe(text)
            st.subheader("Hindi Translation")
            st.write(result[0]["translation_text"])
        else:
            st.warning("Please enter some text.")

elif task == "Zero-shot Classification":
    st.subheader("🎯 Zero-shot Classification")
    text = st.text_area("Enter text", "This is a course about AI and machine learning.")
    labels_input = st.text_input(
        "Candidate labels (comma-separated)",
        "education, politics, sports, technology"
    )
    multi_label = st.checkbox("Multi-label classification", value=False)

    if st.button("Classify (Zero-shot)"):
        if text.strip() and labels_input.strip():
            labels = [l.strip() for l in labels_input.split(",") if l.strip()]
            pipe = get_zero_shot_pipe()
            result = pipe(text, candidate_labels=labels, multi_label=multi_label)
            st.json(result)
        else:
            st.warning("Please enter text and at least one label.")

elif task == "Image Classification":
    st.subheader("🖼️ Image Classification")
    uploaded_file = st.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption="Uploaded Image", use_column_width=True)

        if st.button("Classify Image"):
            pipe = get_image_cls_pipe()
            result = pipe(image)
            st.subheader("Predictions")
            st.json(result)

elif task == "Speech to Text":
    st.subheader("🎙️ Speech to Text (ASR)")
    st.write("Upload an audio file (e.g., .wav, .mp3).")
    uploaded_audio = st.file_uploader("Upload audio", type=["wav", "mp3", "flac", "ogg", "m4a"])

    if uploaded_audio is not None:
        st.audio(uploaded_audio)

        if st.button("Transcribe"):
            # Save to temp file and send path to pipeline
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_audio.name)[1]) as tmp:
                tmp.write(uploaded_audio.read())
                tmp_path = tmp.name

            pipe = get_asr_pipe()
            result = pipe(tmp_path)
            st.subheader("Transcription")
            st.write(result["text"])

            # cleanup temp file
            os.remove(tmp_path)

elif task == "Feature Extraction (Embeddings)":
    st.subheader("📐 Feature Extraction / Embeddings")
    text = st.text_area("Enter text", "This is an example sentence for embeddings.")

    if st.button("Get Embeddings"):
        if text.strip():
            pipe = get_embedding_pipe()
            emb = pipe(text)
            # emb shape: [1, seq_len, hidden_dim]
            seq_len = len(emb[0])
            hidden_dim = len(emb[0][0])
            st.write(f"Embedding shape: [1, {seq_len}, {hidden_dim}]")
            st.markdown("**First token vector (truncated):**")
            st.write(emb[0][0][:10])
        else:
            st.warning("Please enter some text.")

elif task == "Text to Image (Stable Diffusion)":
    st.subheader("🎨 Text to Image (Stable Diffusion)")

    if not torch.cuda.is_available():
        st.warning("CUDA GPU not detected. This will be VERY slow on CPU, or may run out of memory.")

    prompt = st.text_area("Enter image prompt", "a cute robot teaching data science, digital art, vibrant colors")
    steps = st.slider("Inference steps", 10, 50, 25)
    guidance = st.slider("Guidance scale", 1.0, 15.0, 7.5)

    if st.button("Generate Image"):
        if prompt.strip():
            with st.spinner("Generating image... (first time may take a while)"):
                pipe, device = get_sd_pipe()
                image = pipe(
                    prompt,
                    num_inference_steps=steps,
                    guidance_scale=guidance
                ).images[0]
            st.image(image, caption="Generated Image", use_column_width=True)
        else:
            st.warning("Please enter a prompt.")
```

---

## ▶️ 3. Run the App

In the folder with `app.py`:

```bash
streamlit run app.py
```

Then open the local URL (usually `http://localhost:8501`).

---

If you want, I can next:

* add **model selection dropdown per task** (e.g., different sentiment models),
* or export this as a **zip-ready project structure** (with `app.py`, `requirements.txt`, `README.md`).


Alright, let’s build a **free image generation setup using Hugging Face + Transformers** 😎

I’ll show you two options:

1. **Completely free & easiest** → Use **Hugging Face Inference API** (no GPU needed, runs on HF servers)
2. **Local free (needs GPU/strong CPU)** → Download model and run via `transformers` / `diffusers`

Since you wrote *“using tranfrmer”*, I’ll keep it focused on Hugging Face ecosystem (Transformers-style pipelines).

---

## 1️⃣ Setup – Hugging Face Account & Token (One Time)

1. Go to Hugging Face → create a free account.
2. Go to **Settings → Access Tokens** → create a **Read token**.
3. Install required libraries:

```bash
pip install transformers torch pillow requests
```

If you want to use diffusion models (recommended for images):

```bash
pip install diffusers accelerate safetensors
```

---

## 2️⃣ Easiest Way: Use Hugging Face Inference API (Free Tier)

Here we **don’t download** the model; we just call it using HTTPS.
Example using a Stable Diffusion model (text → image):

```python
import requests

API_URL = "https://api-inference.huggingface.co/models/stabilityai/stable-diffusion-2-1"
HF_TOKEN = "YOUR_HF_API_TOKEN"   # <--- paste your token here

headers = {"Authorization": f"Bearer HF_TOKEN_REPLACE"}

def generate_image(prompt, output_path="generated.png"):
    payload = {"inputs": prompt}
    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return

    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Saved image to {output_path}")


if __name__ == "__main__":
    prompt = "a futuristic city at sunset, ultra realistic, 4k"
    generate_image(prompt)
```

👉 **Important:** Replace

```python
headers = {"Authorization": f"Bearer HF_TOKEN_REPLACE"}
```

with

```python
headers = {"Authorization": f"Bearer " + HF_TOKEN}
```

This gives you a **free image generator** using a Hugging Face model, no GPU on your side.

---

## 3️⃣ Using `diffusers` + Transformers Locally (Open-Source, Free)

This uses a **Transformers-style pipeline** but via the `diffusers` library (Hugging Face official for diffusion models).

```python
from diffusers import StableDiffusionPipeline
import torch

# Download any free text-to-image model, example:
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # use float32 if you don't have GPU
)

# If you have a GPU (recommended)
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

prompt = "a cute robot teaching data science, digital art, vibrant colors"
image = pipe(prompt).images[0]

image.save("robot_ds.png")
print("Image saved as robot_ds.png")
```

If you **don’t** have GPU, change to:

```python
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float32
).to("cpu")
```

⚠️ CPU will be **very slow** but still free.

---

## 4️⃣ “Transformers” Style Pipeline (Non-diffusers models)

Some image models are available directly via `transformers` pipelines (e.g., text-to-image, image-to-image). Example:

```python
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch

# Example: Phi-3-Vision / Qwen-VL etc. can generate images or reason about them
# (Many models are more multimodal than pure generators)
```

But for **pure image generation**, **diffusers** is the normal + recommended way in HF ecosystem, and it’s built to work **with transformers under the hood**.

---

## 5️⃣ Wrap It as a Simple “Free HF Image Generator” Function

Here’s a neat utility that automatically uses GPU if available:

```python
from diffusers import StableDiffusionPipeline
import torch

class HFImageGenerator:
    def __init__(self, model_id="runwayml/stable-diffusion-v1-5"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        dtype = torch.float16 if self.device == "cuda" else torch.float32

        self.pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=dtype,
        ).to(self.device)

    def generate(self, prompt: str, file_name: str = "output.png"):
        image = self.pipe(prompt).images[0]
        image.save(file_name)
        print(f"✅ Saved: {file_name}")

if __name__ == "__main__":
    gen = HFImageGenerator()
    gen.generate("a fantasy landscape with floating islands, matte painting", "fantasy.png")
```

This is basically a **free image generator model using Hugging Face** that you can plug into Flask, FastAPI, Streamlit, etc.

---

## 6️⃣ If You Tell Me Your Goal…

For example:

* “Use it in **Flask API**”
* “Use it in **Streamlit app**”
* “Use it in a **Jupyter Notebook** as a teaching demo”

…I can give you **full project code** (app.py + requirements.txt + sample prompts) tailored to that.

For now, you can copy-paste any of the above blocks and run them directly.


## Text generation with GPT-2

In [ ]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

prompt = 'Artificial intelligence is'
inputs = tokenizer(prompt, return_tensors='pt')
output = model.generate(inputs['input_ids'], max_length=50, temperature=0.7, top_p=0.9)
print(tokenizer.decode(output[0], skip_special_tokens=True))


## Text classification (BERT pipeline)

In [ ]:
classifier = pipeline('text-classification', model='bert-base-uncased')
print(classifier('I love using AI for automation!'))


## Translation (T5) using pipeline

In [ ]:
translator = pipeline('translation_en_to_de', model='t5-small')  # or use 't5-small'
print(translator('How are you?'))
